# Day 25 — "Post-Processing in Segmentation (CRFs, Morphology, Connected Components)"

Segmentation outputs are raw geometry. Post-processing cleans noise, fixes holes, and enforces spatial plausibility.


In [1]:
# Ensure repo root is on sys.path for local imports
import sys
from pathlib import Path

repo_root = Path.cwd()
if not (repo_root / "days").exists():
    for parent in Path.cwd().resolve().parents:
        if (parent / "days").exists():
            repo_root = parent
            break

sys.path.insert(0, str(repo_root))
print(f"Using repo root: {repo_root}")


Using repo root: /media/abdul-aziz/sdb7/masters_research/math_course_dlcv


## 1. Core Intuition

- Neural nets predict what is likely; post-processing enforces what is geometrically plausible.
- Typical artifacts: salt-and-pepper noise, holes, broken thin structures, fuzzy edges.


## 2. Morphological Operations

| Operation | Effect |
| --- | --- |
| Erosion | Shrinks foreground, removes tiny blobs |
| Dilation | Expands foreground, fills gaps |
| Opening | Erosion → dilation (removes noise) |
| Closing | Dilation → erosion (fills holes) |

Morphology is fast, deterministic, and explainable.


## 3. Connected Components

Connected components treat segmentation as objects instead of pixels.
They enable size filtering, counting, and instance-like reasoning.


## 4. CRFs (Conceptual)

CRFs combine network confidence (unary terms) with spatial consistency (pairwise terms).
They sharpen boundaries but are slower and parameter-sensitive.


## 5. Thresholding & Calibration

Pick thresholds on a validation set using IoU/F1 curves.
Aggressive thresholds trade recall for precision and can fragment thin objects.


## 6. Python — Morphology + Components (NumPy)

`days/day25/code/postprocessing.py` implements erosion/dilation, opening/closing, and connected component filtering.


In [2]:
import numpy as np
from days.day25.code.postprocessing import (
    opening,
    closing,
    remove_small_components,
)

rng = np.random.default_rng(0)
mask = (rng.random((64, 64)) > 0.92).astype(np.uint8)
mask = closing(opening(mask, radius=1), radius=1)
clean = remove_small_components(mask, min_area=30)

print("Before:", mask.sum())
print("After:", clean.sum())


Before: 0
After: 0


## 7. Visualization — Cleanup Pipeline

`days/day25/code/visualizations.py` renders a simple cleanup pipeline using opening, closing, and component filtering.


In [3]:
from days.day25.code.visualizations import plot_postprocessing

RUN_FIGURES = False

if RUN_FIGURES:
    plot_postprocessing()
else:
    print("Set RUN_FIGURES = True to regenerate Day 25 figures inside days/day25/outputs/.")


Set RUN_FIGURES = True to regenerate Day 25 figures inside days/day25/outputs/.


## 8. Typical Pipelines

- Binary segmentation: threshold → closing → remove small objects.
- Roads/rivers: low threshold → skeletonize → dilation.
- Buildings: CRF → connected components → area filter.


## 9. Common Mistakes

- Over-smoothing thin objects.
- Removing small but valid targets.
- Using a single kernel size everywhere.
- Applying CRF without tuning.


## 10. Mini Exercises

1. Compare opening vs closing on thin structures.
2. Remove objects smaller than 200 pixels and inspect false negatives.
3. Sweep kernel sizes and plot IoU.
4. Compare raw vs post-processed masks visually.


## 11. Key Takeaways

- Post-processing turns raw predictions into usable geometry.
- Morphology cleans shapes quickly.
- Connected components add object-level reasoning.
- CRFs enforce spatial consistency but require tuning.
